In [1]:
!pip install kaggle

! kaggle datasets download -d kreeshrajani/3k-conversations-dataset-for-chatbot

Dataset URL: https://www.kaggle.com/datasets/kreeshrajani/3k-conversations-dataset-for-chatbot
License(s): other
  0% 0.00/67.1k [00:00<?, ?B/s]
100% 67.1k/67.1k [00:00<00:00, 48.7MB/s]


In [24]:
! unzip /content/3k-conversations-dataset-for-chatbot.zip

Archive:  /content/3k-conversations-dataset-for-chatbot.zip
  inflating: Conversation.csv        


In [25]:
import pandas as pd
import numpy as np

In [50]:
data = pd.read_csv('/content/Conversation.csv', index_col=0)
data

,question,answer
0,"hi, how are you doing?",i'm fine. how about yourself?
1,i'm fine. how about yourself?,i'm pretty good. thanks for asking.
2,i'm pretty good. thanks for asking.,no problem. so how have you been?
3,no problem. so how have you been?,i've been great. what about you?
4,i've been great. what about you?,i've been good. i'm in school right now.
...,...,...
3720,that's a good question. maybe it's not old age.,are you right-handed?
3721,are you right-handed?,yes. all my life.
3722,yes. all my life.,you're wearing out your right hand. stop using...
3723,you're wearing out your right hand. stop using...,but i do all my writing with my right hand.


In [51]:
data['question'] = "[Q] " + data['question']
data['answer'] = "[A] " + data['answer']

text = ""

for i in range(len(data)):
    text+=data.iloc[i]["question"]+"\n"
    text+=data.iloc[i]["answer"]
    text+="\n\n"

    # if(i == 1000):
    #   print(i)
    #   break

open("input.txt","w+").write(text)

277430

In [52]:
!pip install transformers
!pip install -U PyPDF2
!pip install python-docx

In [53]:
import pandas as pd
import numpy as np
import re
from PyPDF2 import PdfReader
import os
import docx

In [54]:
# Functions to read different file types

def read_txt(file_path):
    with open(file_path, "r") as file:
        text = file.read()
    return text

def read_documents_from_directory(directory):
    combined_text = ""
    combined_text += read_txt(directory)
    return combined_text


In [55]:

# Read documents from the directory
#train_directory = '/content/drive/MyDrive/ColabNotebooks/data/chatbot_docs/training_data/full_text'
# train_directory = '/content/drive/MyDrive/ColabNotebooks/data/chatbot_docs/training_data/q_and_a'
train_directory = 'input.txt'
text_data = read_documents_from_directory(train_directory)
text_data = re.sub(r'\n+', '\n', text_data).strip()  # Remove excess newline characters

In [56]:
text_data[:200]

open("train.txt","w+").write(text_data)

273704

In [57]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [58]:
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset


In [59]:
def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator

In [60]:
def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)

  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )

  trainer.train()
  trainer.save_model()

In [61]:
#train_file_path = "/content/drive/MyDrive/ColabNotebooks/data/chatbot_docs/combined_text/full_text/train.txt"
train_file_path = "/content/train.txt"
model_name = 'gpt2'
#output_dir = '/content/drive/MyDrive/ColabNotebooks/models/chat_models/custom_full_text'
output_dir = '/content/model/custom_q_and_a'
overwrite_output_dir = False
per_device_train_batch_size = 45
num_train_epochs = 50.0
save_steps = 50000

In [62]:
# Train
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss


In [66]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer

In [83]:
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer


def clean_text(text):
    return text[4:].partition("[Q]")[0]

def generate_text(model_path, sequence, max_length):

    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.90,
    )

    print(clean_text(tokenizer.decode(final_outputs[0], skip_special_tokens=True)))

In [88]:
model2_path = "/content/model/custom_q_and_a"
sequence2 = "[Q] where do like candy or cake"
max_len = 50
generated_text = generate_text(model2_path, sequence2, max_len)

where do like candy or cake come from?
[A] i always buy chocolate or hazelnut oil.

